## Testing notebook
<br/>
it aims to unit test each files and functions and provide a good tuto of the code.
<br/>  
<br/>
For each files the config.py contains a dictionnary to modify the parameters

### Overview :   
- Dataloaders/ : Loading data + transformations :
    - MRI_loader.py 
    - registration_loader.py
- process/ : 
    - losses.py
    - utils.py
- Model.py
- Trainer.py
- main.py


<br/>   
 
### Dataloaders/ : Loading data + transformations :
To upload the data, add transformations and create validation and training dataset.

In [1]:
from config import dataloader_config
dataloader_config

{'rootdir': 'Documents/Projet_3A/data/2D_data/MedNIST/Hand/',
 'batch_size': 1,
 'valid_ratio': 0.2,
 'num_workers': 1,
 'transformation': Compose(
     ToTensor()
 ),
 'vectorize': False}

- rootdir : the root of the folder containing the data
- batch_size 
- valid_ratio : the proportion of validation data you want to include
- num_workers : in case of calcul parallelisation 
- transformation : sequence of torch or monai transformations to apply to the data (rotations, normalize etc..)
note that for 3D vectorized data you absolutely need to use the monai transformations AddChannel
- vectorize : wheter to vectorize or not the data
<br/>

#### MRI_loader.py

In [2]:
from Dataloaders.MRI_loader import get_labels_list
print(get_labels_list(dataloader_config['rootdir']))

FileNotFoundError: [Errno 2] No such file or directory: 'Documents/Projet_3A/data/2D_data/MedNIST/Hand/'

In [3]:
from Dataloaders.MRI_loader import get_data_dicts

partition=get_data_dicts(dataloader_config['rootdir'],dataloader_config['valid_ratio'])
print(len(partition['train']),'\n',len(partition['validation']),'\n',partition['train'][8])

28 
 8 
 /home/luther/Documents/Projet_3A/data/L2R_2021_Task3_test/mask/OASIS_OAS1_0018_MR1


load.py is the file you want to update for differents dataset : take two path as inputs (one for the folder with the fixed image and its segmentation and one for the folder with the moving image and its segmentation), according to your dataset type you might want to modify how the data is load.

In [4]:
from Dataloaders.MRI_loader import load

fixed_image,moving_image,fixed_mask,moving_mask=load(partition['train'][0],partition['train'][1],vectorize=False)
fixed_image.shape

(160, 192, 224)

We also can see above if to_vector from process.utils works
### registration_loader
<br/>
Use the function from MRI_loader.py to create custom pytorch dataset and dataloaders
<br/>

In [5]:
from Dataloaders.registration_loader import Registration_dataset
from torchvision import transforms
from monai.transforms import AddChannel


t = transforms.Compose([transforms.ToTensor(), AddChannel()])
            
dataset=Registration_dataset(partition['train'], t, vectorize=False)
fixed_image, moving_image, fixed_mask, moving_mask=dataset[2]
fixed_image.shape

torch.Size([1, 224, 160, 192])

In [6]:
from Dataloaders.registration_loader import get_dataloaders
training_generator, validation_generator=get_dataloaders(dataloader_config)
next(iter(training_generator))[0].shape

torch.Size([1, 1, 224, 160, 192])

La fonction addchannel de monai permet d'ajouter la dimension channel sans faire un .view dans le training pour des données 3D (inutile pour des données 2D).
<br/>

### Process

#### Utils
utils.py contained a bunch of functions : graphic one, vectorization (with the transformations from the paper) etc.. Only the to_vector is test here (refer to the result notebook or utils.py to learn more).

In [7]:
from process.utils import to_vector
#Currently 2D data but with 3D how would it works ?
fixed_image,moving_image,fixed_mask,moving_mask=load(partition['train'][0],partition['train'][1])
to_vector(fixed_image).shape

(160, 192, 224, 3)

### Losses 
<br/>

#### compute dice test
Dice is a metric between two segmentations.

In [8]:
from process.losses import compute_dice

#let's compare 2 mask from two random image of our dataset
fixed_image,moving_image,fixed_mask,moving_mask=next(iter(training_generator))
compute_dice(fixed_mask, moving_mask)

tensor([[0.4990, 0.6892, 0.6315, 0.3801]])

MSE

In [9]:
from process.losses import MSE

#let's compare 2 images from dataset
MSE(fixed_image, moving_image)

tensor(0.0042, dtype=torch.float64)

Gradient

In [10]:
from process.losses import Grad
import torch

flow_field_random=torch.abs(torch.randn([1,3,160,192,224])) 
Grad('l2').loss(flow_field_random)

tensor(0.7267)

In [4]:
from config import criterion_config
criterion_config

{'MSE': 1, 'Grad': {'Norm': 'l2', 'weight': 0.05}, 'Dice': None}

criterion_config is used to configure the model loss (through the Model.py file)
- MSE : the weight of MSE in the loss 
- Grad : 'l2' or 'l1' penalization + the weight in the loss
- Dice : Dice is compute anyway but you can add it to the loss or not by putting a weight instead of None
### Model

<br/>


In [3]:
from config import model_config
model_config

{'inshape': (224, 160, 192),
 'nb_unet_features': [[16, 32, 32], [32, 16, 16]],
 'src_feats': 1,
 'trg_feats': 1}

- inshape : the data input shape (2D or 3D)
- nb_unet_features : the size of the encoder layers and of the decoder layers
- src_feats/trg_feats : as you might have read in the vxm tuto, theses features handles the number of inputs channel (refer to the end of the vxm tuto)

In [5]:
from config import scheduler_config
scheduler_config

{'scheduler': 'ROP',
 'mode': 'min',
 'factor': 0.2,
 'patience': 10,
 'threshold': 1e-05,
 'verbose': True}

A loss scheduler is implemented (aims to reduce the loss when its not decreasing for a time)
- scheduler : type of scheduler, only Reduce On Plateau is implemented
- mode : decreasing
- factor : new_loss=factor x loss
- patience : decrease loss if its not decreasing for 10 epochs
- treshold : decrease treshold

In [11]:
#model initialization
from Model import model
from config import model_config, criterion_config

test_model=model(model_config, criterion_config)
test_model.net

VxmDense(
  (unet_model): Unet(
    (upsample): Upsample(scale_factor=2.0, mode=nearest)
    (downarm): ModuleList(
      (0): ConvBlock(
        (main): Conv3d(2, 16, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
        (activation): LeakyReLU(negative_slope=0.2)
      )
      (1): ConvBlock(
        (main): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
        (activation): LeakyReLU(negative_slope=0.2)
      )
      (2): ConvBlock(
        (main): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
        (activation): LeakyReLU(negative_slope=0.2)
      )
      (3): ConvBlock(
        (main): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
        (activation): LeakyReLU(negative_slope=0.2)
      )
    )
    (uparm): ModuleList(
      (0): ConvBlock(
        (main): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (activation): LeakyReLU(negative_slope=0.

In [ ]:
#model forward
fixed_image,moving_image=fixed_image.to(test_model.device),moving_image.to(test_model.device)
pred_image,flow_field=test_model.net(moving_image.float(), fixed_image.float())
pred_image.shape,flow_field.shape

In [14]:
#model interpolation
moving_mask=moving_mask.to(test_model.device)
pred_mask = test_model.net.transformer(moving_mask.float(), flow_field)
pred_mask.shape

NameError: name 'flow_field' is not defined

In [13]:
#model criterion
import torch
flow_field_random=torch.abs(torch.randn([1,3,160,192,224])) 
loss, train_mean_dice = test_model.criterion(moving_image, moving_mask, fixed_image, fixed_mask, flow_field_random)
print(loss, train_mean_dice)

tensor(0.0436, dtype=torch.float64) tensor([0.5087, 0.7257, 0.7022, 0.3913])


### Trainer

In [6]:
from config import train_config
train_config

{'nb_epochs': 1000,
 'checkpoints_path': 'Documents/Projet_3A/VectorMorph/models/',
 'verbose': True,
 'checkpoint': 50}

- nb_epochs : number of training epoch
- checkpoints_path : path where saving the training weights
- checkpoint : how much epoch to wait for saving weights

In [1]:
from torch import load, save 
from Model import model
from Trainer import Trainer
from config import model_config, dataloader_config, train_config

model = model(model_config)
trainer = Trainer(model, dataloader_config, train_config)
trainer.trainloader

In [2]:
trainer.train()

100%|███████████████████████████████████████████| 28/28 [00:02<00:00, 11.63it/s]


(0.20536147917010963, tensor(2.2776))

In [3]:
trainer.test()

100%|█████████████████████████████████████████████| 8/8 [00:00<00:00, 33.48it/s]


(0.07549482911870577, tensor(2.1523))

In [4]:
trainer.verbose()


Train Loss................: 0.00
Test Loss.................: 0.00
Train Mean Dice............: 0.00
Test Mean Dice.............: 0.00

Best Test Mean Dice........: 0.00


In [5]:
trainer.update_state()

100%|█████████████████████████████████████████████| 8/8 [00:00<00:00, 19.76it/s]


In [6]:
trainer.verbose()


Train Loss................: 0.19
Test Loss.................: 0.07
Train Mean Dice............: 0.00
Test Mean Dice.............: 0.00

Best Test Mean Dice........: 2.81


### Main.py
test with only 3 epoch

In [ ]:
train_config['nb_epochs']=3
from main import main, evaluate
main(model_config, dataloader_config, train_config, criterion_config, scheduler_config) #can be launch directly from a terminal

In [ ]:
evaluate(train_config['checkpoints_path']+'', model_config, dataloader_config, train_config, criterion_config, scheduler_config)